In [ ]:
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import GeometryCollection

# enable KML support
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

### Read kml files

In [ ]:
strecke = gpd.read_file('../data/ISR-ISR_V_GEO_TEN_KLASSIFIZIERUNG.kml')
bahnhöfe = gpd.read_file('../data/ISR-ISR_V_GEO_TEN_KLASSIFIZIERUNG.kml')

In [ ]:
strecke = strecke.drop(['Description'], axis=1)

In [ ]:
geo = strecke.loc[0,'geometry']

transform 
```python
    GeometryCollection[
        GeometryCollection[
            ...
        ]
    ]
```
to 
```python
    GeometryCollection[
        ...
    ]
```

In [ ]:
for i in range(len(strecke)):
    new_geo =  []
    for g in strecke.iat[i, 1]:
        if type(g) == shapely.geometry.collection.GeometryCollection:
            for u in g:
                new_geo.append(u)
                geo.intersection(u)
        else:
            new_geo.append(g)
    strecke.iat[i, 1] = GeometryCollection(new_geo)

In [ ]:
strecke.plot(figsize=(21*2, 24*2))

In [ ]:
strecke.plot(figsize=(21*2, 24*2))

In [ ]:
bahnhöfe.plot(figsize=(21, 24))